# Quantize Model

### Import Packages

In [ ]:
import tensorflow as tf
# Local imports
from ml_project_util.path import path_definition
from ml_project_util.flatten_model import flatten_condtitional
from ml_project_util.quantization_util import quant_weights, quant_activations, quant_model, quant_bw_search

### Variable Paths, Execution Environments

In [ ]:
# dict = path_definition(ds_rel_path='catsdogsconv/CatsDogs')
dict = path_definition()
BASE_PATH = dict['BASE_PATH']
PATH_DATASET = dict['PATH_DATASET']
PATH_TEST = dict['PATH_TEST']
PATH_RAWDATA = dict['PATH_RAWDATA']
PATH_JOINEDDATA = dict['PATH_JOINEDDATA']
PATH_SAVEDMODELS = dict['PATH_SAVEDMODELS']

### Load Model

In [ ]:
# Two first letters for dataset, next letter stands for parameter
# 3 numbers are for the number of epochs, and the last 4 digits are for validation loss
# e.g. CD4_P2_FT_003_val0.0336
model_name = ''
parent_folder = model_name[:3]
filepath = f'{PATH_SAVEDMODELS}/{parent_folder}/{model_name}.keras'
model = tf.keras.models.load_model(filepath)
model = flatten_condtitional(model, model_name)

### Quantize & Evaluate

In [ ]:
qw_model = quant_weights(model, model_name, num_bits=8, mode='quant')
qwa_model =  quant_activations(model, model_name, num_bits=8, mode='eval', design='hw')

In [ ]:
qwa_model =  quant_model(model, model_name, num_bits=8, mode='eval', design='hw')

In [ ]:
qwa_model = quant_bw_search(model, model_name, range(7,17))

### Save Metrics

In [ ]:
import json

# save accuracy in json in
bitwidth = 8
mode = "qw"
short_name = model_name[:-10]
accuracy_path = f'{BASE_PATH}/Docs_Reports/Quant/{short_name}_acc_bw_{mode}.json'
# read json
try:
    with open(accuracy_path, 'r') as file:
        acc_bw_dict = json.load(file)
except:
    print('No accuracy per bitwidth json file found in specified path!')
    acc_bw_dict = {}
# add in dict
acc_bw_dict[f'{bitwidth}b'] = { "accuracy": 0.97840, "loss": 0.11877}
# write json
with open(accuracy_path, 'w') as f:
    json.dump(acc_bw_dict, f, indent=4)

### Save qw model

In [ ]:
# same path + qw subfolder
parent_folder = model_name[:3]
short_name = model_name[:-10]
model_path = f'{PATH_SAVEDMODELS}/{parent_folder}/Quant/{short_name}_qw.keras'
model.save(model_path)